# Part 2: Introduce Movie Ratings dataset and join it with Oscar Awards (working with my group)

In [61]:
import pandas as pd

# Load the datasets related to ~45,000 
movies_metadata = pd.read_csv('movies_metadata.csv')
links = pd.read_csv('links.csv')

/var/folders/xf/159f6qbs1lb9cc3jn9z7g4j80000gn/T/ipykernel_13967/1672647590.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_metadata.csv')


In [62]:
# Perform ETL procedures for movies_metadata

# Convert release_date to date format and extract year only
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')
movies_metadata['release_year'] = movies_metadata['release_date'].dt.year

# Drop all irrelevant fields
movies_metadata = movies_metadata[['id', 'title', 'revenue', 'popularity', 'release_year']]

# Drop all records that don't have values in 'title' and 'release_year'
movies_metadata.dropna(subset=['title', 'release_year'], inplace=True)

# Convert the 'id' column to numeric type and drop non-numeric values
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
movies_metadata.dropna(subset=['id'], inplace=True)

# Make movie titles lower case
movies_metadata['title'] = movies_metadata['title'].str.lower()

movies_metadata

,id,title,revenue,popularity,release_year
0,862,toy story,373554033.0,21.946943,1995.0
1,8844,jumanji,262797249.0,17.015539,1995.0
2,15602,grumpier old men,0.0,11.7129,1995.0
3,31357,waiting to exhale,81452156.0,3.859495,1995.0
4,11862,father of the bride part ii,76578911.0,8.387519,1995.0
...,...,...,...,...,...
45460,30840,robin hood,0.0,5.683753,1991.0
45462,111109,century of birthing,0.0,0.178241,2011.0
45463,67758,betrayal,0.0,0.903007,2003.0
45464,227506,satan triumphant,0.0,0.003503,1917.0


In [63]:
# Join movies_metadata and links dataframes

# Convert the 'tmdbId' column to numeric type and drop non-numeric values
links['tmdbId'] = pd.to_numeric(links['tmdbId'], errors='coerce')
links.dropna(subset=['tmdbId'], inplace=True)

# Drop duplicates and keep only 'tmdbId' and 'movieId' columns
links.drop_duplicates(inplace=True)
links = links[['tmdbId', 'movieId']]

# Inner join movies_metadata and links dataframes based on 'id' and 'tmdbId' columns, respectively
movies_data_links = pd.merge(movies_metadata, links, left_on='id', right_on='tmdbId')

# Drop any values from links that are not present in movies_metadata
movies_data_links.dropna(subset=['movieId'], inplace=True)

movies_data_links

,id,title,revenue,popularity,release_year,tmdbId,movieId
0,862,toy story,373554033.0,21.946943,1995.0,862.0,1
1,8844,jumanji,262797249.0,17.015539,1995.0,8844.0,2
2,15602,grumpier old men,0.0,11.7129,1995.0,15602.0,3
3,31357,waiting to exhale,81452156.0,3.859495,1995.0,31357.0,4
4,11862,father of the bride part ii,76578911.0,8.387519,1995.0,11862.0,5
...,...,...,...,...,...,...,...
45433,30840,robin hood,0.0,5.683753,1991.0,30840.0,176267
45434,111109,century of birthing,0.0,0.178241,2011.0,111109.0,176271
45435,67758,betrayal,0.0,0.903007,2003.0,67758.0,176273
45436,227506,satan triumphant,0.0,0.003503,1917.0,227506.0,176275


In [64]:
# Load the ratings dataset
ratings = pd.read_csv('ratings.csv')

# Drop fields 'userId' and 'timestamp'
ratings.drop(['userId', 'timestamp'], axis=1, inplace=True)

# Convert field 'rating' into a numeric value with 2 decimal points
ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce').round(2)

# Group all records by field 'movieId', values for 'rating' should be average
ratings = ratings.groupby('movieId').mean().reset_index()

ratings

,movieId,rating
0,1,3.888157
1,2,3.236953
2,3,3.175550
3,4,2.875713
4,5,3.079565
...,...,...
45110,176267,4.000000
45111,176269,3.500000
45112,176271,5.000000
45113,176273,1.000000


In [65]:
# Join movies_data_links and ratings data
movies_ratings = pd.merge(movies_data_links, ratings, on='movieId')

# Drop any records that are not present in both data frames
movies_ratings.dropna(subset=['revenue', 'rating'], inplace=True)

# Drop all id fields that are no longer required
movies_ratings.drop(['tmdbId', 'movieId', 'id'], axis=1, inplace=True)

movies_ratings

,title,revenue,popularity,release_year,rating
0,toy story,373554033.0,21.946943,1995.0,3.888157
1,jumanji,262797249.0,17.015539,1995.0,3.236953
2,grumpier old men,0.0,11.7129,1995.0,3.175550
3,waiting to exhale,81452156.0,3.859495,1995.0,2.875713
4,father of the bride part ii,76578911.0,8.387519,1995.0,3.079565
...,...,...,...,...,...
44713,caged heat 3000,0.0,0.661558,1995.0,2.000000
44714,robin hood,0.0,5.683753,1991.0,4.000000
44715,century of birthing,0.0,0.178241,2011.0,5.000000
44716,betrayal,0.0,0.903007,2003.0,1.000000


# 3. Which film has received the most nominations & awards in a ceremony?

In [66]:
# Load the Oscar data
oscar_data = pd.read_csv('the_oscar_award.csv')

# Convert 'film' and 'category' fields to lower case
oscar_data['film'] = oscar_data['film'].str.lower()
oscar_data['category'] = oscar_data['category'].str.lower()
oscar_data

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,actor,Richard Barthelmess,the noose,False
1,1927,1928,1,actor,Emil Jannings,the last command,True
2,1927,1928,1,actress,Louise Dresser,a ship comes in,False
3,1927,1928,1,actress,Janet Gaynor,7th heaven,True
4,1927,1928,1,actress,Gloria Swanson,sadie thompson,False
...,...,...,...,...,...,...,...
10754,2022,2023,95,writing (original screenplay),Written by Martin McDonagh,the banshees of inisherin,NaN
10755,2022,2023,95,writing (original screenplay),Written by Daniel Kwan & Daniel Scheinert,everything everywhere all at once,NaN
10756,2022,2023,95,writing (original screenplay),Written by Steven Spielberg & Tony Kushner,the fabelmans,NaN
10757,2022,2023,95,writing (original screenplay),Written by Todd Field,tár,NaN


In [67]:
oscar_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10759 entries, 0 to 10758
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year_film      10759 non-null  int64 
 1   year_ceremony  10759 non-null  int64 
 2   ceremony       10759 non-null  int64 
 3   category       10759 non-null  object
 4   name           10756 non-null  object
 5   film           10449 non-null  object
 6   winner         10639 non-null  object
dtypes: int64(3), object(4)
memory usage: 588.5+ KB


In [68]:
# Summarize the awards/nominations of films for all years.
oscar_data_summary = oscar_data.groupby(by=['film', 'category']).agg({'winner': 'sum', 'name': 'count'}).reset_index()
oscar_data_summary = oscar_data_summary.rename(columns={'name': 'Nominations'})
oscar_data_summary['Awards'] = oscar_data_summary['winner'].apply(lambda x:int(x))
del oscar_data_summary['winner']
oscar_data_summary = oscar_data_summary.groupby(by='film').sum().sort_values(by='Awards',ascending=False)

print(oscar_data_summary.head(10))

                                               Nominations  Awards
film                                                              
titanic                                                 16      12
ben-hur                                                 12      11
west side story                                         18      11
the lord of the rings: the return of the king           11      11
the last emperor                                         9       9
the english patient                                     12       9
gigi                                                     9       9
gone with the wind                                      13       8
on the waterfront                                       12       8
slumdog millionaire                                     10       8


# 4. Do movies with the highest rating in a given year win 'Best Picture' award?

In [69]:
# Filter to only include 'best motion picture' or 'best picture' categories
oscar_data = oscar_data[oscar_data['category'].str.contains('best motion picture') | oscar_data['category'].str.contains('best picture')]

In [70]:
# Join oscar_data and movies_ratings on fields 'title' and 'film' respectively
merged_data = pd.merge(oscar_data, movies_ratings, left_on='film', right_on='title', how='inner')

# Keep only the records present in both data frames
merged_data = merged_data.loc[merged_data['year_film'] == merged_data['release_year']]

# Drop the unnecessary columns
merged_data = merged_data.drop(['year_film', 'film'], axis=1)

merged_data

,year_ceremony,ceremony,category,name,winner,title,revenue,popularity,release_year,rating
0,1945,17,best motion picture,Paramount,False,double indemnity,2500000.0,6.49432,1944.0,4.202604
2,1945,17,best motion picture,Metro-Goldwyn-Mayer,False,gaslight,0.0,14.545058,1944.0,4.031027
4,1945,17,best motion picture,Paramount,True,going my way,16300000.0,5.682778,1944.0,3.572881
5,1945,17,best motion picture,Selznick International Pictures,False,since you went away,0.0,4.505452,1944.0,3.653061
6,1945,17,best motion picture,20th Century-Fox,False,wilson,2000000.0,0.773081,1944.0,3.214286
...,...,...,...,...,...,...,...,...,...,...
515,2017,89,best picture,"Matt Damon, Kimberly Steward, Chris Moore, Lau...",False,manchester by the sea,75026965.0,12.821257,2016.0,3.861973
516,2017,89,best picture,"Adele Romanski, Dede Gardner and Jeremy Kleine...",True,moonlight,65046687.0,14.171675,2016.0,3.775287
517,2018,90,best picture,"Peter Spears, Luca Guadagnino, Emilie Georges ...",False,call me by your name,0.0,4.300874,2017.0,3.500000
519,2018,90,best picture,"Emma Thomas and Christopher Nolan, Producers",False,dunkirk,519876949.0,30.938854,2017.0,4.113475


In [71]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 520
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year_ceremony  378 non-null    int64  
 1   ceremony       378 non-null    int64  
 2   category       378 non-null    object 
 3   name           378 non-null    object 
 4   winner         378 non-null    object 
 5   title          378 non-null    object 
 6   revenue        378 non-null    float64
 7   popularity     378 non-null    object 
 8   release_year   378 non-null    float64
 9   rating         378 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 32.5+ KB


In [72]:
# Group the data by year_ceremony and find the maximum rating for each year
max_ratings = merged_data.groupby('year_ceremony')['rating'].transform(max)

# Create the 'highest_rating' column based on the comparison
merged_data['highest_rating'] = merged_data['rating'] == max_ratings

merged_data

,year_ceremony,ceremony,category,name,winner,title,revenue,popularity,release_year,rating,highest_rating
0,1945,17,best motion picture,Paramount,False,double indemnity,2500000.0,6.49432,1944.0,4.202604,True
2,1945,17,best motion picture,Metro-Goldwyn-Mayer,False,gaslight,0.0,14.545058,1944.0,4.031027,False
4,1945,17,best motion picture,Paramount,True,going my way,16300000.0,5.682778,1944.0,3.572881,False
5,1945,17,best motion picture,Selznick International Pictures,False,since you went away,0.0,4.505452,1944.0,3.653061,False
6,1945,17,best motion picture,20th Century-Fox,False,wilson,2000000.0,0.773081,1944.0,3.214286,False
...,...,...,...,...,...,...,...,...,...,...,...
515,2017,89,best picture,"Matt Damon, Kimberly Steward, Chris Moore, Lau...",False,manchester by the sea,75026965.0,12.821257,2016.0,3.861973,False
516,2017,89,best picture,"Adele Romanski, Dede Gardner and Jeremy Kleine...",True,moonlight,65046687.0,14.171675,2016.0,3.775287,False
517,2018,90,best picture,"Peter Spears, Luca Guadagnino, Emilie Georges ...",False,call me by your name,0.0,4.300874,2017.0,3.500000,False
519,2018,90,best picture,"Emma Thomas and Christopher Nolan, Producers",False,dunkirk,519876949.0,30.938854,2017.0,4.113475,True


In [73]:
# check that the 'highest_rating' values were assigned correctly
merged_data_filtered = merged_data[(merged_data['year_ceremony'] == 1945) | (merged_data['year_ceremony'] == 2017)]
merged_data_filtered

,year_ceremony,ceremony,category,name,winner,title,revenue,popularity,release_year,rating,highest_rating
0,1945,17,best motion picture,Paramount,False,double indemnity,2500000.0,6.49432,1944.0,4.202604,True
2,1945,17,best motion picture,Metro-Goldwyn-Mayer,False,gaslight,0.0,14.545058,1944.0,4.031027,False
4,1945,17,best motion picture,Paramount,True,going my way,16300000.0,5.682778,1944.0,3.572881,False
5,1945,17,best motion picture,Selznick International Pictures,False,since you went away,0.0,4.505452,1944.0,3.653061,False
6,1945,17,best motion picture,20th Century-Fox,False,wilson,2000000.0,0.773081,1944.0,3.214286,False
508,2017,89,best picture,"Shawn Levy, Dan Levine, Aaron Ryder and David ...",False,arrival,203388186.0,30.83786,2016.0,4.022611,True
509,2017,89,best picture,"Scott Rudin, Denzel Washington and Todd Black,...",False,fences,64414761.0,11.599716,2016.0,3.501908,False
510,2017,89,best picture,"Bill Mechanic and David Permut, Producers",False,hacksaw ridge,175302354.0,21.037699,2016.0,3.872426,False
511,2017,89,best picture,"Carla Hacken and Julie Yorn, Producers",False,hell or high water,37589296.0,12.565896,2016.0,3.886400,False
512,2017,89,best picture,"Donna Gigliotti, Peter Chernin, Jenno Topping,...",False,hidden figures,230698791.0,16.816834,2016.0,3.977663,False


In [74]:
# select all 'Best Picture' winners and show their rating
winners = merged_data[merged_data['winner'] == True]
winners = winners[['year_ceremony', 'title', 'highest_rating']]
winners = winners.sort_values(['year_ceremony'])
winners = winners.reset_index(drop=True)
winners

,year_ceremony,title,highest_rating
0,1945,going my way,False
1,1946,the lost weekend,False
2,1947,the best years of our lives,True
3,1948,gentleman's agreement,False
4,1949,hamlet,False
...,...,...,...
63,2012,the artist,False
64,2013,argo,False
65,2014,12 years a slave,False
66,2016,spotlight,True


In [75]:
# How many times in the history of Oscar did a film with the highest rating won?
count_highest_rating = winners['highest_rating'].sum()
print("Total number of years in which top rated film won 'Best Picture':", count_highest_rating)

Total number of years in which top rated film won 'Best Picture': 27


# 5. Does the most popular film in a given year win 'Best Picture' award?

In [76]:
# Convert `popularity` to numeric data type
merged_data['popularity'] = pd.to_numeric(merged_data['popularity'], errors='coerce')

In [77]:
# Group the data by year_ceremony and find the most popular film for each year
max_popularity = merged_data.groupby('year_ceremony')['popularity'].transform(max)

# Create the 'max_popularity' column based on the comparison
merged_data['max_popularity'] = merged_data['popularity'] == max_popularity
merged_data

,year_ceremony,ceremony,category,name,winner,title,revenue,popularity,release_year,rating,highest_rating,max_popularity
0,1945,17,best motion picture,Paramount,False,double indemnity,2500000.0,6.494320,1944.0,4.202604,True,False
2,1945,17,best motion picture,Metro-Goldwyn-Mayer,False,gaslight,0.0,14.545058,1944.0,4.031027,False,True
4,1945,17,best motion picture,Paramount,True,going my way,16300000.0,5.682778,1944.0,3.572881,False,False
5,1945,17,best motion picture,Selznick International Pictures,False,since you went away,0.0,4.505452,1944.0,3.653061,False,False
6,1945,17,best motion picture,20th Century-Fox,False,wilson,2000000.0,0.773081,1944.0,3.214286,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
515,2017,89,best picture,"Matt Damon, Kimberly Steward, Chris Moore, Lau...",False,manchester by the sea,75026965.0,12.821257,2016.0,3.861973,False,False
516,2017,89,best picture,"Adele Romanski, Dede Gardner and Jeremy Kleine...",True,moonlight,65046687.0,14.171675,2016.0,3.775287,False,False
517,2018,90,best picture,"Peter Spears, Luca Guadagnino, Emilie Georges ...",False,call me by your name,0.0,4.300874,2017.0,3.500000,False,False
519,2018,90,best picture,"Emma Thomas and Christopher Nolan, Producers",False,dunkirk,519876949.0,30.938854,2017.0,4.113475,True,False


In [78]:
# select all 'Best Picture' winners and show their revenue
winners_popularity = merged_data[merged_data['winner'] == True]
winners_popularity = winners_popularity[['year_ceremony', 'title', 'max_popularity']]
winners_popularity = winners_popularity.sort_values(['year_ceremony'])
winners_popularity = winners_popularity.reset_index(drop=True)
winners_popularity

,year_ceremony,title,max_popularity
0,1945,going my way,False
1,1946,the lost weekend,True
2,1947,the best years of our lives,False
3,1948,gentleman's agreement,True
4,1949,hamlet,False
...,...,...,...
63,2012,the artist,False
64,2013,argo,False
65,2014,12 years a slave,True
66,2016,spotlight,False


In [79]:
# How many times in the history of Oscar did a film with the highest revenue generated?
count_max_popularity = winners_popularity['max_popularity'].sum()
print("Total number of years in which the most popular film won 'Best Picture':", count_max_popularity)

Total number of years in which the most popular film won 'Best Picture': 28
